In [25]:
import time
import re

In [26]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [27]:
wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver.exe"
addr = "https://news.nate.com/view/20200415n09530?mid=n1008"    # 기사 주소 넣기
addr_nums = re.findall("\d+", addr)
addr_comment = 'http://comm.news.nate.com/Comment/ArticleComment/List?artc_sq='+addr_nums[0]+'n'+addr_nums[1] # 댓글창 주소

In [28]:
driver = webdriver.Chrome(wd)
driver.get(addr_comment)

In [29]:
pages = 2

In [30]:
try:
    for i in range(5):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"body > div.commentBox.reply_hide > div:nth-child(3) > div.paging_wrap > a:nth-child("+str(pages)+")"))).click()
        time.sleep(1)
        print(pages, end=" ")
        pages+=1
except exceptions.ElementNotVisibleException as e: # 페이지 끝
    pass
except Exception as e: # 다른 예외 발생시 확인
    print(e)

2 3 4 5 6 

In [31]:
html = driver.page_source
dom = BeautifulSoup(html, "lxml")

In [32]:
# 댓글이 들어있는 페이지 전체 크롤링
comments_raw = dom.find_all('dd',{'class':'usertxt'})
# 댓글의 text만 뽑는다.
comments = [comment.text for comment in comments_raw]

In [33]:
comments[:10]

['\n완전기대되요 목욜 기다리기가 힘드네요ㅠ\n임영웅 못하는게 먼가요\t\t\t  ',
 '\n넘넘 듣고 싶었던 임영웅 데시파시토!! 모두를 감동시킬거라 확신합니다\t\t\t  ',
 '\n임영웅버전의 데스파시토는 어떨지 너무 기대됩니다~\n무슨노래를 불러도 자기화시키는 신의경지라~\n원곡 65억만뷰의 데스파시토 라틴계음악으로  또\n우리를 어떻게 초토화시킬지 본방사수 설레이네요~\n#임영웅 #사랑의콜센타#응원합니다\t\t\t  ',
 '\n임영웅  춤선이 너무이쁜 가수구요 모든 장르 노래 다 진짜 진짜 잘 부르는 가수입니다\t\t\t  ',
 '\n엄청 기대 됩니다 ~~임영웅 섹시미 포텐 터지겠죠?  \n목요일이 빨리 왔으면......\t\t\t  ',
 '\n사진만봐도 벌써 심장떨려요  ... 얼마나 멋지게 불러주려나 완전기대합니다  임영웅 화이팅~!!!!\t\t\t  ',
 '\n임영웅 도대체 당신의 매력의 끝은 어디인가요~^~\t\t\t  ',
 '\n데스파시토 넘넘 기대됩니다 ~  \n목요일까정 우찌 기다릴꼬...\t\t\t  ',
 '\n날마다더!더!빛내주길바란다 임영웅💓\t\t\t  ',
 '\n바램듣고 노래에 푹빠져서 유틉검색하다 데시파시토 듣는순간 오마이갓..임영웅한테 다시 푹빠진노래..넘넘기대만땅..낼본방사수ㅡㅡㅡ엉능 목욜아 내게오렴..\t\t\t  ']

In [34]:
normalized_text = []
for string in comments:
    try:
        tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ",string)
    except Exception as e:
        print(string)
        break
    normalized_text.append(tokens)
print(normalized_text[:5])

[' 완전기대되요 목욜 기다리기가 힘드네요ㅠ 임영웅 못하는게 먼가요 ', ' 넘넘 듣고 싶었던 임영웅 데시파시토 모두를 감동시킬거라 확신합니다 ', ' 임영웅버전의 데스파시토는 어떨지 너무 기대됩니다 무슨노래를 불러도 자기화시키는 신의경지라 원곡 억만뷰의 데스파시토 라틴계음악으로 또 우리를 어떻게 초토화시킬지 본방사수 설레이네요 임영웅 사랑의콜센타 응원합니다 ', ' 임영웅 춤선이 너무이쁜 가수구요 모든 장르 노래 다 진짜 진짜 잘 부르는 가수입니다 ', ' 엄청 기대 됩니다 임영웅 섹시미 포텐 터지겠죠 목요일이 빨리 왔으면 ']


In [35]:
p_comment_list = []

In [36]:
for comment in comments:
    p_comment_list.append(normalized_text)

In [37]:
file = open('fi_final.txt','w',encoding = 'utf-8')
for comment in normalized_text:
    file.write(comment + '\n')
file.close()